--- 
### 3-13장 Chest X-Ray images 3. Advanced Model  
- https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia 
- 경진대회가 아닌 이미지셋만 주어짐 
- X-ray 이미지로 부터 정상 또는 폐렴 여부를 판별하는 문제 
- train / val /test 폴더에 각각 Normal/Pneumonia 폴더가 있다. (정상/폐렴의 이미지)
- 평가지표로 정확도, 재현율, F1-score 사용 
- efficientnet b1,b2, b3를 앙상블 하여 성능을 개선한다. 
---

In [17]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

file_count = 0  # 출력한 파일 개수를 추적
max_files = 10  # 출력할 파일의 최대 개수

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        file_count += 1
        if file_count >= max_files:
            break
    if file_count >= max_files:
        break

/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/.DS_Store
/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/val/.DS_Store
/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/val/PNEUMONIA/person1947_bacteria_4876.jpeg
/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/val/PNEUMONIA/person1946_bacteria_4875.jpeg
/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/val/PNEUMONIA/person1952_bacteria_4883.jpeg
/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/val/PNEUMONIA/person1954_bacteria_4886.jpeg
/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/val/PNEUMONIA/person1951_bacteria_4882.jpeg
/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/val/PNEUMONIA/person1946_bacteria_4874.jpeg
/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/val/PNEUMONIA/person1949_bacteria_4880.jpeg
/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/val/PNEUMONIA/.DS_Store


In [18]:
# 1. 시드고정 
import torch 
import random 
import numpy as np 
import pandas as pd 
import os

seed = 50 
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.enabled = False

# 2.GPU setting 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [19]:
# 3. 데이터 로딩 
data_path = '/kaggle/input/chest-xray-pneumonia/chest_xray/'
train_path = os.path.join(data_path, 'train/')
val_path = os.path.join(data_path, 'val/')
test_path = os.path.join(data_path, 'test/')

# 4. 이미지 증강을 위한 데이터 변환기 정의 
from torchvision import transforms 

transform_train = transforms.Compose([transforms.Resize((250,250)),   # 이미지 크기를 250x250으로 
                                      transforms.CenterCrop(180),     # 이미지 가운데 부분을 180x180 만큼 잘라냄 
                                      transforms.RandomHorizontalFlip(0.5), # 50% 확률로 수평변환 
                                      transforms.RandomVerticalFlip(0.2),   # 20% 확률로 수직변환 
                                      transforms.RandomRotation(20),        # +/- 20도 랜덤 회전 
                                      transforms.ToTensor(), 
                                      transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))  # RGB 평균&표준편차로 표준화 
                                      ])

transform_test = transforms.Compose([transforms.Resize((250,250)), 
                                      transforms.CenterCrop(180),                                       
                                      transforms.ToTensor(), 
                                      transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
                                      ])

# 5. 데이터셋 생성 
# ImageFolder는 root의 경로에 있는 이미지들을 바로 데이터셋으로 만들어준다. 
from torchvision.datasets import ImageFolder  

dataset_train = ImageFolder(root=train_path, transform=transform_train)
dataset_valid = ImageFolder(root=test_path, transform=transform_test)

# 6. 멀티프로세싱 설정, 제너레이터 시드고정 
def seed_worker(worker_id): 
    worker_seed = torch.initial_seed() % 2**32
    random.seed(worker_seed)
    np.random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(0)

# 7. 데이터로더 생성 
from torch.utils.data import DataLoader
batch_size = 8
loader_train = DataLoader(dataset=dataset_train, batch_size=batch_size, shuffle=True, worker_init_fn=seed_worker, generator=g, num_workers=2 )
loader_valid = DataLoader(dataset=dataset_valid, batch_size=batch_size, shuffle=False, worker_init_fn=seed_worker, generator=g, num_workers=2 )

In [20]:
# 8. efficientnet 모델 임포트 
#pip install efficientnet-pytorch==0.7.1
from efficientnet_pytorch import EfficientNet

model_list = []  # 다수의 모델 사용을 위한, 모델 저장용 리스트 

# 3개의 모델을 생성하고 디바이스에 할당 
# efficientNet b1, b2, b3 모델은 각각 약 65백만, 77백만, 11백만개의 파라미터를 보유 
efficientnet_b1 = EfficientNet.from_pretrained('efficientnet-b1', num_classes=2)
efficientnet_b2 = EfficientNet.from_pretrained('efficientnet-b2', num_classes=2)
efficientnet_b3 = EfficientNet.from_pretrained('efficientnet-b3', num_classes=2)

efficientnet_b1 = efficientnet_b1.to(device)
efficientnet_b2 = efficientnet_b2.to(device)
efficientnet_b3 = efficientnet_b3.to(device)

# 리스트에 모델 저장 
model_list.append(efficientnet_b1)
model_list.append(efficientnet_b2)
model_list.append(efficientnet_b3)

Loaded pretrained weights for efficientnet-b1
Loaded pretrained weights for efficientnet-b2
Loaded pretrained weights for efficientnet-b3


In [21]:
# 9. 손실함수, 옵티마이저 정의 
# 옵티마이저는 AdamW 사용하며, 모델이 3개이므로 각각 정의 필요 
import torch.nn as nn 
criterion = nn.CrossEntropyLoss()

optimizer1 = torch.optim.Adam(model_list[0].parameters(), lr=0.0006, weight_decay=0.001)
optimizer2 = torch.optim.Adam(model_list[1].parameters(), lr=0.0006, weight_decay=0.001)
optimizer3 = torch.optim.Adam(model_list[2].parameters(), lr=0.0006, weight_decay=0.001)

In [22]:
# 10. 스케쥴러 설정 
#pip install transformers
from transformers import get_cosine_schedule_with_warmup 
epochs = 20 

scheduler1 = get_cosine_schedule_with_warmup(optimizer1, 
                                             num_warmup_steps = len(loader_train)*3,
                                             num_training_steps = len(loader_train)*epochs )

scheduler2 = get_cosine_schedule_with_warmup(optimizer2, 
                                             num_warmup_steps = len(loader_train)*3,
                                             num_training_steps = len(loader_train)*epochs )

scheduler3 = get_cosine_schedule_with_warmup(optimizer3, 
                                             num_warmup_steps = len(loader_train)*3,
                                             num_training_steps = len(loader_train)*epochs )

In [23]:
# 11-0. 훈련, 검증, 최적 가중치 찾기를 위함 함수 개발 
# 에폭 수만큼 훈련과 검증을 반복하면서 최적 모델의 가중치를 찾아서 마지막에 반환하는 구조다.
# 데이터 로더를 이용하므로 훈련과 검증을 미니배치 단위로 수행한다.

from sklearn.metrics import accuracy_score # 정확도 계산 함수
from sklearn.metrics import recall_score   # 재현율 계산 함수
from sklearn.metrics import f1_score       # F1 점수 계산 함수
from tqdm.notebook import tqdm             # 진행률 표시 막대

def train(model, loader_train, loader_valid, criterion, optimizer, 
          scheduler=None, epochs=10, save_file='model_state_dict.pth'):
    
    valid_loss_min = np.inf # 최소 손실값 초기화 (검증 데이터용) 

    # 총 에폭만큼 반복
    for epoch in range(epochs):
        print(f'에폭 [{epoch+1}/{epochs}] \n-----------------------------')
        
        # == [ 훈련 ] ==============================================
        model.train()        # 모델을 훈련 상태로 설정
        epoch_train_loss = 0 # 에폭별 손실값 초기화 (훈련 데이터용)
        # '반복 횟수'만큼 반복 
        for images, labels in tqdm(loader_train):
            # 이미지, 레이블(타깃값) 데이터 미니배치를 장비에 할당 
            images = images.to(device)
            labels = labels.to(device)
            
            # 옵티마이저 내 기울기 초기화
            optimizer.zero_grad()
            # 순전파 : 이미지 데이터를 신경망 모델의 입력값으로 사용해 출력값 계산
            outputs = model(images)
            # 손실 함수를 활용해 outputs와 labels의 손실값 계산
            loss = criterion(outputs, labels)
            # 현재 배치에서의 손실 추가 (훈련 데이터용)
            epoch_train_loss += loss.item() 
            loss.backward()       # 역전파 수행
            optimizer.step()      # 가중치 갱신
            if scheduler != None: # 스케줄러 학습률 갱신 
                scheduler.step() 

        # 훈련 데이터 손실값 출력
        print(f'\t훈련 데이터 손실값 : {epoch_train_loss/len(loader_train):.4f}')
        
        # == [ 검증 ] ==============================================
        model.eval()         # 모델을 평가 상태로 설정 
        epoch_valid_loss = 0 # 에폭별 손실값 초기화 (검증 데이터용)
        preds_list = []      # 예측값 저장용 리스트 초기화
        true_list = []       # 실젯값 저장용 리스트 초기화
        
        with torch.no_grad(): # 기울기 계산 비활성화
            for images, labels in loader_valid:
                images = images.to(device)
                labels = labels.to(device)
                
                outputs = model(images)
                loss = criterion(outputs, labels)
                epoch_valid_loss += loss.item()
                
                # 예측값 및 실제값 
                preds = torch.max(outputs.cpu(), dim=1)[1].numpy() 
                true = labels.cpu().numpy() 
    
                preds_list.extend(preds)
                true_list.extend(true)
                
        # 정확도, 재현율, F1 점수 계산
        val_accuracy = accuracy_score(true_list, preds_list)
        val_recall = recall_score(true_list, preds_list)
        val_f1_score = f1_score(true_list, preds_list)

        # 검증 데이터 손실값 및 정확도, 재현율, F1점수 출력
        print(f'\t검증 데이터 손실값 : {epoch_valid_loss/len(loader_valid):.4f}')
        print(f'\t정확도 : {val_accuracy:.4f} / 재현율 : {val_recall:.4f} / F1 점수 : {val_f1_score:.4f}')
        # == [ 최적 모델 가중치 찾기 ] ==============================
        # 현 에폭에서의 손실값이 최소 손실값 이하면 모델 가중치 저장 
        if epoch_valid_loss <= valid_loss_min: 
            print(f'\t### 검증 데이터 손실값 감소 ({valid_loss_min:.4f} --> {epoch_valid_loss:.4f}). 모델 저장')
            # 모델 가중치를 파일로 저장 
            torch.save(model.state_dict(), save_file) 
            valid_loss_min = epoch_valid_loss # 최소 손실값 갱신 
    return torch.load(save_file) # 저장한 모델 가중치를 불러와 반환

In [24]:
# 11-1. 모델훈련 - efficientnet_b1 & 최적 가중치 저장 
model_state_dict = train(model=model_list[0], 
                         loader_train=loader_train, 
                         loader_valid=loader_valid,
                         criterion=criterion, 
                         optimizer=optimizer1, 
                         scheduler=scheduler1, 
                         epochs=epochs)

model_list[0].load_state_dict(model_state_dict)

에폭 [1/20] 
-----------------------------


  0%|          | 0/652 [00:00<?, ?it/s]

	훈련 데이터 손실값 : 0.3133
	검증 데이터 손실값 : 0.4585
	정확도 : 0.8446 / 재현율 : 0.9974 / F1 점수 : 0.8891
	### 검증 데이터 손실값 감소 (inf --> 35.7656). 모델 저장
에폭 [2/20] 
-----------------------------


  0%|          | 0/652 [00:00<?, ?it/s]

	훈련 데이터 손실값 : 0.1467
	검증 데이터 손실값 : 0.4033
	정확도 : 0.8686 / 재현율 : 0.9692 / F1 점수 : 0.9021
	### 검증 데이터 손실값 감소 (35.7656 --> 31.4588). 모델 저장
에폭 [3/20] 
-----------------------------


...

	훈련 데이터 손실값 : 0.0404
	검증 데이터 손실값 : 0.5328
	정확도 : 0.8526 / 재현율 : 0.9974 / F1 점수 : 0.8943
에폭 [20/20] 
-----------------------------


  0%|          | 0/652 [00:00<?, ?it/s]

	훈련 데이터 손실값 : 0.0398
	검증 데이터 손실값 : 0.4779
	정확도 : 0.8702 / 재현율 : 0.9974 / F1 점수 : 0.9057


<ipython-input-23-1189ba53dd16>:81: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(save_file) # 저장한 모델 가중치를 불러와 반환


<All keys matched successfully>

In [25]:
# 11-2. 모델훈련 - efficientnet_b2 & 최적 가중치 저장 
model_state_dict = train(model=model_list[1], 
                         loader_train=loader_train, 
                         loader_valid=loader_valid,
                         criterion=criterion, 
                         optimizer=optimizer2, 
                         scheduler=scheduler2, 
                         epochs=epochs)

model_list[1].load_state_dict(model_state_dict)

에폭 [1/20] 
-----------------------------


  0%|          | 0/652 [00:00<?, ?it/s]

	훈련 데이터 손실값 : 0.3041
	검증 데이터 손실값 : 0.3671
	정확도 : 0.8606 / 재현율 : 0.9974 / F1 점수 : 0.8994
	### 검증 데이터 손실값 감소 (inf --> 28.6333). 모델 저장
에폭 [2/20] 
-----------------------------


  0%|          | 0/652 [00:00<?, ?it/s]

	훈련 데이터 손실값 : 0.1501
	검증 데이터 손실값 : 1.0558
	정확도 : 0.7356 / 재현율 : 1.0000 / F1 점수 : 0.8254
에폭 [3/20] 
-----------------------------


...

	훈련 데이터 손실값 : 0.0460
	검증 데이터 손실값 : 0.4007
	정확도 : 0.8686 / 재현율 : 0.9923 / F1 점수 : 0.9042
에폭 [20/20] 
-----------------------------


  0%|          | 0/652 [00:00<?, ?it/s]

	훈련 데이터 손실값 : 0.0414
	검증 데이터 손실값 : 0.4112
	정확도 : 0.8638 / 재현율 : 0.9923 / F1 점수 : 0.9010


<ipython-input-23-1189ba53dd16>:81: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(save_file) # 저장한 모델 가중치를 불러와 반환


<All keys matched successfully>

In [26]:
# 11-3. 모델훈련 - efficientnet_b3 & 최적 가중치 저장 
model_state_dict = train(model=model_list[2], 
                         loader_train=loader_train, 
                         loader_valid=loader_valid,
                         criterion=criterion, 
                         optimizer=optimizer3, 
                         scheduler=scheduler3, 
                         epochs=epochs)

model_list[2].load_state_dict(model_state_dict)

에폭 [1/20] 
-----------------------------


  0%|          | 0/652 [00:00<?, ?it/s]

	훈련 데이터 손실값 : 0.2732
	검증 데이터 손실값 : 0.3926
	정확도 : 0.8718 / 재현율 : 0.9897 / F1 점수 : 0.9061
	### 검증 데이터 손실값 감소 (inf --> 30.6264). 모델 저장
에폭 [2/20] 
-----------------------------


  0%|          | 0/652 [00:00<?, ?it/s]

	훈련 데이터 손실값 : 0.1478
	검증 데이터 손실값 : 0.5861
	정확도 : 0.8606 / 재현율 : 0.9974 / F1 점수 : 0.8994
에폭 [3/20] 
-----------------------------


...

	훈련 데이터 손실값 : 0.0427
	검증 데이터 손실값 : 0.4697
	정확도 : 0.8542 / 재현율 : 0.9949 / F1 점수 : 0.8950
에폭 [20/20] 
-----------------------------


  0%|          | 0/652 [00:00<?, ?it/s]

	훈련 데이터 손실값 : 0.0396
	검증 데이터 손실값 : 0.5116
	정확도 : 0.8542 / 재현율 : 0.9974 / F1 점수 : 0.8953


<ipython-input-23-1189ba53dd16>:81: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(save_file) # 저장한 모델 가중치를 불러와 반환


<All keys matched successfully>

In [29]:
# 12. 훈련된 모델을 사용하여 테스트 데이터에 대한 예측 준비 (테스트 데이터셋, 데이터 로더 생성, 예측함수)  
dataset_test = ImageFolder(root=test_path, transform=transform_test)
loader_test = DataLoader(dataset=dataset_test, batch_size=batch_size, shuffle=False, worker_init_fn=seed_worker, generator=g, num_workers=2)

def predict(model, loader_test, return_true=False): 
    model.eval() 
    preds_list=[]
    true_list=[]

    with torch.no_grad(): 
        for images, labels in loader_test: 
            images=images.to(device)
            labels=labels.to(device)
            outputs = model(images)

            preds = torch.max(outputs.cpu(), dim=1)[1].numpy()
            preds_list.extend(preds)

            true = labels.cpu().numpy()
            true_list.extend(true)

    if return_true: 
        return true_list, preds_list
    else: 
        return preds_list

In [30]:
# 13. 예측 1~3 & 실제/예측값 저장 
true_list, preds_list1 = predict(model=model_list[0], loader_test=loader_test, return_true=True)

## 2~3번째 예측에서 또 true값을 구할 필요는 없으니 ... 
preds_list2 = predict(model=model_list[1], loader_test=loader_test)
preds_list3 = predict(model=model_list[2], loader_test=loader_test)

In [32]:
# 14. 개별모델 예측 결과확인 
print('########### efficientnet_b1 ################')
print('정확도 = {:.4f}'.format(accuracy_score(true_list, preds_list1)))
print('재현율 = {:.4f}'.format(accuracy_score(true_list, preds_list1)))
print('f1 score = {:.4f}'.format(accuracy_score(true_list, preds_list1)))

print('########### efficientnet_b2 ################')
print('정확도 = {:.4f}'.format(accuracy_score(true_list, preds_list2)))
print('재현율 = {:.4f}'.format(accuracy_score(true_list, preds_list2)))
print('f1 score = {:.4f}'.format(accuracy_score(true_list, preds_list2)))

print('########### efficientnet_b3 ################')
print('정확도 = {:.4f}'.format(accuracy_score(true_list, preds_list3)))
print('재현율 = {:.4f}'.format(accuracy_score(true_list, preds_list3)))
print('f1 score = {:.4f}'.format(accuracy_score(true_list, preds_list3)))

########### efficientnet_b1 ################
정확도 = 0.9038
재현율 = 0.9038
f1 score = 0.9038
########### efficientnet_b2 ################
정확도 = 0.8718
재현율 = 0.8718
f1 score = 0.8718
########### efficientnet_b3 ################
정확도 = 0.8942
재현율 = 0.8942
f1 score = 0.8942


In [34]:
# 15. 3개 모델 앙상블 : 3개 예측값의 평균을 구한다. 
ensemble_preds = []
for i in range(len(preds_list1)):
    pred_element = np.round((preds_list1[i] + preds_list2[i] + preds_list3[i])/3)
    ensemble_preds.append(pred_element)

# 16. 앙상블 예측 결과확인 
print('########### Ensemble of efficientnet_b1~b3 ################')
print('정확도 = {:.4f}'.format(accuracy_score(true_list, ensemble_preds)))
print('재현율 = {:.4f}'.format(accuracy_score(true_list, ensemble_preds)))
print('f1 score = {:.4f}'.format(accuracy_score(true_list, ensemble_preds)))

########### Ensemble of efficientnet_b1~b3 ################
정확도 = 0.9071
재현율 = 0.9071
f1 score = 0.9071
